In [55]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import calendar
from scipy.stats import chi2_contingency
from IPython.display import display
import os, re
project_folder =os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
decade = 'Q34_1'
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data_general = pd.read_csv(f'{project_folder}/data/process/final_geo_coding.csv', skiprows=[1, 2]).reset_index()
data_general['Q34_1'] = data_general['Q34'].dropna().map(lambda x:int(x/10)*10)
data_general

,level_0,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,StartDate,EndDate,Status,IPAddress,Progress,...,Q40,Q40_3_TEXT,lat,lon,score,stolen_bikes_place,country,city,state,Q34_1
0,0,2,718,745,745,5/3/2022 19:05,5/3/2022 19:19,IP Address,208.98.223.42,100,...,"A bicycle registry/recovery system (e.g., Bike...",NaN,-113.490140,53.545450,100.0,"Edmonton,Alberta,CAN",CAN,Edmonton,Alberta,1930.0
1,1,3,313,323,323,4/27/2022 16:30,4/27/2022 16:44,IP Address,99.171.110.253,100,...,Email,NaN,-117.874460,33.749580,100.0,"Santa Ana,California,USA",USA,Santa Ana,California,1930.0
2,2,4,1409,1492,1492,5/11/2022 14:54,5/11/2022 15:00,IP Address,76.103.138.149,100,...,Email,NaN,-121.885420,37.338650,100.0,"San Jose,California,USA",USA,San Jose,California,1940.0
3,3,5,1603,1710,1710,5/15/2022 20:21,5/15/2022 20:27,IP Address,68.149.96.52,100,...,Email,NaN,-113.490140,53.545450,100.0,"Edmonton,Alberta,CAN",CAN,Edmonton,Alberta,1940.0
4,4,6,5,7,7,4/22/2022 11:46,4/22/2022 11:53,IP Address,97.113.1.186,100,...,Email,NaN,-122.329450,47.603570,100.0,"Seattle,Washington,USA",USA,Seattle,Washington,1940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,1816,1818,1773,1967,1967,5/16/2022 17:58,5/16/2022 18:03,IP Address,73.189.49.55,82,...,NaN,NaN,-122.285410,38.299110,100.0,"Napa,California,USA",USA,Napa,California,NaN
1817,1817,1819,1786,1981,1981,6/1/2022 14:44,6/1/2022 14:51,IP Address,72.194.49.254,100,...,Email,NaN,-119.728880,34.441470,100.0,"Santa Barbara,California,USA",USA,Santa Barbara,California,NaN
1818,1818,1820,1795,1990,1990,6/2/2022 14:51,6/2/2022 14:56,IP Address,107.122.81.38,100,...,Other (please specify):,Facebook,-119.699050,34.419380,100.0,"Santa Barbara,California,USA",USA,Santa Barbara,California,NaN
1819,1819,1821,1823,2021,2021,5/24/2022 12:17,5/24/2022 12:20,IP Address,174.90.223.242,69,...,NaN,NaN,-114.057141,51.045113,100.0,"Calgary,Alberta,CAN",CAN,Calgary,Alberta,NaN


In [73]:
# show the questions
pd.read_csv(f'{project_folder}/data/process/new_data.csv', skiprows=[2]).reset_index().iloc[0]

index                                                          0
StartDate                                             Start Date
EndDate                                                 End Date
Status                                             Response Type
IPAddress                                             IP Address
                                     ...                        
Q38            What is the highest level of education you hav...
Q39            What do you consider your main ethnic origin o...
Q39_10_TEXT    What do you consider your main ethnic origin o...
Q40            Where did you hear about this survey? - Select...
Q40_3_TEXT     Where did you hear about this survey? - Other ...
Name: 0, Length: 75, dtype: object

In [80]:
questions ={'Q2':'part','Q6':'day_time','Q7':'is_locked','Q8':'locked_type','Q9':'theft_location','Q10':'is_regi','Q11':'report_loc','Q12':'is_insured','Q13':'estimate_val','Q14':'is_electric','Q15':'bike_type','Q17':'is_university','Q18':'is_recovered','Q19':'is_online','Q21':'is_police_assist','Q23':'recovery_loc','Q24':'con_recovery','Q34': 'birth', 'Q35': 'gender', 'Q36': 'income', 'Q38': 'education', 'Q39': 'ethnic_origin'}

In [81]:
def get_the_highest(table_updated, table_to_agg):
# This method aggregates small numbers of response types while maintaining 10% percentiles
    if table_to_agg[percentage].sum()<10:
        if len(table_to_agg)==1:
            # No aggregation is required
            table_updated = table_updated.append(table_to_agg)
        else:
            table_updated.loc['Aggregated Responses'] = round(table_to_agg.sum(), 1)
        return table_updated
    else:
        table_updated = table_updated.append(table_to_agg.iloc[0])
        return get_the_highest(table_updated, table_to_agg.iloc[1:])


In [82]:
def sub_aggregate_def(indices:list,j,sum)-> list:
    """
    add all the response types that together are less than 10%
    :param indices: list to updata
    :param j: next response types index to check
    :param sum: the current percentile sum
    :return:
    """
    temp_sum = sub_aggregate.iloc[j][percentage] +sum
    if temp_sum>10:
        return indices
    else:
        indices.append(j)
        if j+1 <len_sub_aggregate:
            indices = sub_aggregate_def(indices,j+1,temp_sum)
    return indices


In [100]:
new_tables = {}
percentage = 'percentage'
# Specific code for birth year


for pair in questions.items():
    # statistic by count (display also by percentage)
    # pair = ('Q39','ethnic_origin')
    question = pair[0]
    #  code for birth year has already conducted
    if question =='Q34':
        table_0 = DataFrame(data_general.groupby('Q34').count()['level_0'].sort_values(ascending=False).rename('count'))
        # convert year to decade.
        decade = 'decade'
        table_0[decade] = table_0.index.map(lambda x:int(x/10)*10)
        table = DataFrame(table_0.groupby(decade).sum()['count'].sort_values(ascending=False))
        table[percentage] = round(table['count']/table['count'].sum()*100,1)
        new_tables['Q34_1_birth'] = table.sort_values(by=decade)
        continue

    # work on specific question
    table_0 = DataFrame(data_general.groupby(question).count()['level_0'].sort_values(ascending=False).rename('count'))
    # Clean unnecessary data and white space
    table = table_0.set_index(table_0.index.map(lambda x: re.sub("\(.*?\)","",x)).str.replace('      ','').str.strip())

    table[percentage] = round(table['count']/table['count'].sum()*100,1)
    # Aggregate responses type with less than 10 responses
    table_aggregate = table[table[percentage]<10]
    table_new = table.copy()

    if len(table_aggregate)>1:
         # There are more than 1 values with less than 10% respondent
        table_new =table[table[percentage]>10]
        table_add = get_the_highest(DataFrame(columns=table_aggregate.columns),table_aggregate)
        # There are more than 1 values with less than 10% respondents that not part of aggregation process before
        if len(table_add)>2:
            sub_aggregate = table_add.iloc[:-1].sort_values(by=percentage)
            i =0
            len_sub_aggregate = len(sub_aggregate)
            while i < len_sub_aggregate:
                if i ==len_sub_aggregate -1:
                    indices = [i]
                else:
                    indices = sub_aggregate_def([i],i+1,sub_aggregate.iloc[i][percentage])
                # aggregate sll the responses type based on indices
                temp_df =sub_aggregate.iloc[indices]
                table_new.loc[' OR '.join(list(temp_df.index))] = temp_df.sum()
                i = indices[-1]+1
            table_new =table_new.append(table_add.iloc[-1])
        else:
            table_new = table_new.append(table_add)

    # add the new table into tables dictionary
    new_tables[f'{question}_{pair[1]}'] = table_new.sort_values(by=percentage,ascending=False)
    # break
with pd.ExcelWriter(f"{project_folder}/python/notebooks/output/general/output.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables.items()]

C:\Users\Achituv\AppData\Local\Temp\ipykernel_19140\3906830916.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_new = table_new.append(table_add)
C:\Users\Achituv\AppData\Local\Temp\ipykernel_19140\3031605931.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_updated.append(table_to_agg.iloc[0])
C:\Users\Achituv\AppData\Local\Temp\ipykernel_19140\3031605931.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_updated.append(table_to_agg.iloc[0])
C:\Users\Achituv\AppData\Local\Temp\ipykernel_19140\3031605931.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_

In [99]:
# Run code without aggregation
new_tables = {}
percentage = 'percentage'
# Specific code for birth year
for pair in questions.items():
    # statistic by count (display also by percentage)
    # pair = ('Q39','ethnic_origin')
    question = pair[0]
    #  code for birth year has already conducted
    if question =='Q34':
        table_0 = DataFrame(data_general.groupby('Q34').count()['level_0'].sort_values(ascending=False).rename('count'))
        # convert year to decade.
        table_0[decade] = table_0.index.map(lambda x:int(x/10)*10)
        table = DataFrame(table_0.groupby(decade).sum()['count'].sort_values(ascending=False))
        table[percentage] = round(table['count']/table['count'].sum()*100,1)
        new_tables['Q34_1_birth'] = table.sort_values(by=decade)
        continue

    # work on specific question
    table_0 = DataFrame(data_general.groupby(question).count()['level_0'].sort_values(ascending=False).rename('count'))
    # Clean unnecessary data and white space
    table = table_0.set_index(table_0.index.map(lambda x: re.sub("\(.*?\)","",x)).str.replace('      ','').str.strip())

    table[percentage] = round(table['count']/table['count'].sum()*100,1)

    # add the new table into tables dictionary
    new_tables[f'{question}_{pair[1]}'] = table.sort_values(by=percentage,ascending=False)
    # break
with pd.ExcelWriter(f"{project_folder}/python/notebooks/output/general/no_agg.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables.items()]

In [85]:
# United States VS Canada
new_tables = {}
percentage = 'percentage'
countries_names = list(data_general['country'].drop_duplicates())

countries_data = {countries_names[0]:data_general[data_general['country']==countries_names[0]],
                  countries_names[1]:data_general[data_general['country']==countries_names[1]]}

for pair in questions.items():
    by_countries_table = DataFrame(columns=countries_names)
    # statistic by count (display also by percentage)
    # pair = ('Q34', 'birth')
    question = pair[0]
    #  code for birth year has already conducted
    if question =='Q34':
        for item in countries_names:
            country_data= countries_data[item]
            # Specific code for birth year
            table_0 = DataFrame(country_data.groupby('Q34').count()['level_0'].sort_values(ascending=False).rename('count'))
            # convert year to decade.
            decade = 'Q34_1'
            table_0[decade] = table_0.index.map(lambda x:int(x/10)*10)
            table = DataFrame(table_0.groupby(decade).sum()['count'].sort_values(ascending=False))
            by_countries_table[item] = round(table['count']/table['count'].sum()*100,1)

        by_countries_table.columns = by_countries_table.columns + '(%)'
        by_countries_table.fillna(0,inplace=True)
        new_tables[f'{question}_{pair[1]}'] = by_countries_table.sort_values(by=decade)
        continue
    for item in countries_names:
        country_data= countries_data[item]
        # work on specific question
        table_0 = DataFrame(country_data.groupby(question).count()['level_0'].sort_values(ascending=False).rename('count'))
        # Clean unnecessary data and white space
        table = table_0.set_index(table_0.index.map(lambda x: re.sub("\(.*?\)","",x)).str.replace('      ','').str.strip())
        by_countries_table[item] = round(table['count']/table['count'].sum()*100,1)
    by_countries_table.columns = by_countries_table.columns + '(%)'
    by_countries_table.fillna(0,inplace=True)
        # add the new table into tables dictionary
    new_tables[f'{question}_{pair[1]}'] = by_countries_table.sort_values(by='USA(%)',ascending=False)
    # break
with pd.ExcelWriter(f"{project_folder}/python/notebooks/output/general/usa_canada.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables.items()]

In [ ]:
def bivariate_analysis(main_vars:dict, secondary_vars:dict,name_analysis:str):
    """
    It goes over the dictionary and calculate the cross relation between the variable inside
    :param name_analysis: it is used to create new folder if necessary
    :param main_vars:
    :param secondary_vars:
    :return:
    """
    res_path = f'{data_path}{name_analysis}'
    os.makedirs(res_path,exist_ok = True)
    with pd.ExcelFile(f"{data_path}no_agg.xlsx") as reader:
        stat = {item[0]:pd.read_excel(reader, sheet_name=f'{item[0]}_{item[1]}') for item in main_vars.items()}
    for main_var in main_vars.items():
        new_tables_data = {}
        for other in secondary_vars.items():
            print(f'{other}\t',end="")
            # GroupBy two questions (one of which is main_var)
            table_00 = DataFrame(data_general.groupby([main_var[0], other[0]]).count()['level_0'].rename(''))
            # Get the data in readable structure
            table_0 =table_00.unstack(fill_value=0)
            # Clean columns and index
            table_0.columns = table_0.columns.droplevel().map(lambda x: re.sub("\(.*?\)","",str(x))).str.replace('      ','').str.strip()
            table_1 = table_0.set_index(table_0.index.map(lambda x: re.sub("\(.*?\)","",str(x))).str.replace('      ','').str.strip())

            # get percentage (questions variables for each group)
            old_columns =table_1.columns
            table_2 = table_1.copy()
            print(main_var[0])
            sum = stat[main_var[0]].set_index(main_var[0])['count']
            table_2[old_columns +' (%)'] = round(table_1.div(sum, axis=0)*100,1)
            table_2['sum'] = sum
            new_tables_data[f'{other[0]}_{other[1]}'] =table_2
        with pd.ExcelWriter(f"{res_path}/{main_var[0]}_{main_var[1]}.xlsx") as writer:
            [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables_data.items()]

In [52]:
def bivariate_analysis_update(data_to_group:DataFrame, main_vars_to_group:list, secondary_vars:list, field_to_count:str, my_data_path:str, ):
    r"""
    It goes over the dictionary and calculate the cross relation between the variable inside
    :param my_data_path: he code creates a new folder here if it does not already exist, and it saves the new Excel files there.
    :param analysis_name: T
    :param is_staty: to use data from other data saved on the disk
    :param data_to_group: data to work on
    :param field_to_count: which field to retain after groupby and counting
    :param name_analysis: it is used to create new folder if necessary
    :param main_vars_to_group:
    :param secondary_vars:
    :return:
    """
    os.makedirs(my_data_path,exist_ok = True)
    for main_var in main_vars_to_group:
        col_name,print_main = check_type(main_var)
        print(f'\n{print_main}')
        new_tables_data = {}
        data1 = data_to_group[~data_to_group[col_name].isin(['No Response', 'no response'])].dropna(subset= col_name)
        for other in secondary_vars:
            col_secondary,print_secondary = check_type(other)
            print(f'{print_secondary}\t',end="")
            data2 = data1[~data_to_group[col_secondary].isin(['No Response', 'no response'])].dropna(subset=col_secondary)
            sum_field = data2.groupby(col_name).count()[field_to_count].rename('count')
            # GroupBy two questions (one of which is main_var)
            table_00 = DataFrame(data2.groupby([col_name, col_secondary]).count()[field_to_count].rename(''))
            # Get the data in readable structure
            table_0 =table_00.unstack(fill_value=0)
            # Clean columns and index
            table_0.columns = table_0.columns.droplevel()

            # get percentage (questions variables for each group)
            old_columns =table_0.columns
            table_2 = table_0.copy()
            table_2[old_columns.astype(str) +' (%)'] = round(table_0.div(sum_field, axis=0) * 100, 1)
            table_2['sum_field'] = sum_field
            new_tables_data[f'{print_secondary}'] =table_2.sort_values(by='sum_field',ascending=False)
        with pd.ExcelWriter(f"{my_data_path}/{print_main}.xlsx") as writer:
            [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables_data.items()]

In [56]:
data_path = f'{project_folder}/python/notebooks/output/general/'
# find correlation between two variables - Demographic

demographic_data= {'Q34_1': 'birth', 'Q35': 'gender', 'Q36': 'income', 'Q38': 'education', 'Q39': 'ethnic_origin'}
other_data ={'Q2': 'part', 'Q6': 'day_time', 'Q7': 'is_locked', 'Q8': 'locked_type', 'Q9': 'theft_location', 'Q10': 'is_regi', 'Q11': 'report_loc', 'Q12': 'is_insured', 'Q13': 'estimate_val', 'Q14': 'is_electric', 'Q15': 'bike_type', 'Q17': 'is_university','Q18':'is_recovered', 'Q19': 'is_online', 'Q21': 'is_police_assist', 'Q23': 'recovery_loc', 'Q24': 'con_recovery'}

demographic_data= [item for item in demographic_data.items()]
other_data  = [item for item in other_data.items()]
bivariate_analysis_update(data_to_group=data_general,main_vars_to_group=demographic_data,secondary_vars=other_data,field_to_count='level_0',my_data_path=f'{data_path}/demographic')


# find correlation between two variables -Recovery

recovery_data= {'Q18':'is_recovered', 'Q19': 'is_online', 'Q21': 'is_police_assist', 'Q23': 'recovery_loc', 'Q24': 'con_recovery'}
other_data ={'Q2': 'part', 'Q6': 'day_time', 'Q7': 'is_locked', 'Q8': 'locked_type', 'Q9': 'theft_location', 'Q10': 'is_regi', 'Q11': 'report_loc', 'Q12': 'is_insured', 'Q13': 'estimate_val', 'Q14': 'is_electric', 'Q15': 'bike_type', 'Q17': 'is_university','Q34_1': 'birth', 'Q35': 'gender', 'Q36': 'income', 'Q38': 'education', 'Q39': 'ethnic_origin'}
recovery_data= [item for item in recovery_data.items()]
other_data  = [item for item in other_data.items()]
bivariate_analysis_update(data_to_group=data_general,main_vars_to_group=recovery_data,secondary_vars=other_data,field_to_count='level_0',my_data_path=f'{data_path}/recovery')




Q34_1_birth
Q2_part	Q6_day_time	Q7_is_locked	Q8_locked_type	Q9_theft_location	Q10_is_regi	Q11_report_loc	Q12_is_insured	Q13_estimate_val	Q14_is_electric	Q15_bike_type	Q17_is_university	Q18_is_recovered	Q19_is_online	Q21_is_police_assist	Q23_recovery_loc	Q24_con_recovery	
Q35_gender
Q2_part	Q6_day_time	Q7_is_locked	Q8_locked_type	Q9_theft_location	Q10_is_regi	Q11_report_loc	Q12_is_insured	Q13_estimate_val	Q14_is_electric	Q15_bike_type	Q17_is_university	Q18_is_recovered	Q19_is_online	Q21_is_police_assist	Q23_recovery_loc	Q24_con_recovery	
Q36_income
Q2_part	Q6_day_time	Q7_is_locked	Q8_locked_type	Q9_theft_location	Q10_is_regi	Q11_report_loc	Q12_is_insured	Q13_estimate_val	Q14_is_electric	Q15_bike_type	Q17_is_university	Q18_is_recovered	Q19_is_online	Q21_is_police_assist	Q23_recovery_loc	Q24_con_recovery	
Q38_education
Q2_part	Q6_day_time	Q7_is_locked	Q8_locked_type	Q9_theft_location	Q10_is_regi	Q11_report_loc	Q12_is_insured	Q13_estimate_val	Q14_is_electric	Q15_bike_type	Q17_is_universit

In [48]:
# This code run the aalysis on the cities with the highest number of answers
data_general.groupby('city').count()
data_general_city = data_general[data_general['city'].isin(['San Francisco','Seattle','Calgary','Edmonton','Los Angeles'])]
data_general_city

,level_0,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,StartDate,EndDate,Status,IPAddress,Progress,...,Q40,Q40_3_TEXT,lat,lon,score,stolen_bikes_place,country,city,state,Q34_1
0,0,2,718,745,745,5/3/2022 19:05,5/3/2022 19:19,IP Address,208.98.223.42,100,...,"A bicycle registry/recovery system (e.g., Bike...",NaN,-113.490140,53.545450,100.0,"Edmonton,Alberta,CAN",CAN,Edmonton,Alberta,1930.0
3,3,5,1603,1710,1710,5/15/2022 20:21,5/15/2022 20:27,IP Address,68.149.96.52,100,...,Email,NaN,-113.490140,53.545450,100.0,"Edmonton,Alberta,CAN",CAN,Edmonton,Alberta,1940.0
4,4,6,5,7,7,4/22/2022 11:46,4/22/2022 11:53,IP Address,97.113.1.186,100,...,Email,NaN,-122.329450,47.603570,100.0,"Seattle,Washington,USA",USA,Seattle,Washington,1940.0
6,6,8,1214,1277,1277,5/11/2022 11:00,5/11/2022 11:04,IP Address,73.231.228.78,100,...,Email,NaN,-122.419640,37.777120,100.0,"San Francisco,California,USA",USA,San Francisco,California,1940.0
9,9,11,604,630,630,5/3/2022 14:19,5/3/2022 14:27,IP Address,50.99.114.237,100,...,Email,NaN,-113.490140,53.545450,100.0,"Edmonton,Alberta,CAN",CAN,Edmonton,Alberta,1940.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,1813,1815,1762,1952,1952,5/13/2022 12:47,5/13/2022 12:51,IP Address,136.25.110.98,82,...,NaN,NaN,-122.419640,37.777120,100.0,"San Francisco,California,USA",USA,San Francisco,California,NaN
1814,1814,1816,1765,1956,1956,5/14/2022 18:15,5/14/2022 18:17,IP Address,72.134.228.254,69,...,NaN,NaN,-118.245450,34.053570,100.0,"Los Angeles,California,USA",USA,Los Angeles,California,NaN
1815,1815,1817,1768,1961,1961,5/15/2022 22:05,5/15/2022 22:09,IP Address,24.17.91.3,69,...,NaN,NaN,-122.329450,47.603570,100.0,"Seattle,Washington,USA",USA,Seattle,Washington,NaN
1819,1819,1821,1823,2021,2021,5/24/2022 12:17,5/24/2022 12:20,IP Address,174.90.223.242,69,...,NaN,NaN,-114.057141,51.045113,100.0,"Calgary,Alberta,CAN",CAN,Calgary,Alberta,NaN


In [41]:
def check_type(value_check):
    r"""
    if it is tuple the name and the column names are different
    :param value_check:
    :return:
    """
    return (value_check[0], '_'.join(value_check)) if isinstance(value_check, tuple) else (value_check, value_check)

In [42]:
check_type(('Q6', 'day_time'))

('Q6', 'Q6_day_time')

In [53]:
data_path = f'{project_folder}/python/notebooks/output/general/'
# analysis by the four city
city_data= ['city']
other_data ={'Q2': 'part', 'Q6': 'day_time', 'Q7': 'is_locked', 'Q8': 'locked_type', 'Q9': 'theft_location', 'Q10': 'is_regi', 'Q11': 'report_loc', 'Q12': 'is_insured', 'Q13': 'estimate_val', 'Q14': 'is_electric', 'Q15': 'bike_type', 'Q17': 'is_university','Q18':'is_recovered', 'Q19': 'is_online', 'Q21': 'is_police_assist', 'Q23': 'recovery_loc', 'Q24': 'con_recovery','Q34_1': 'birth', 'Q35': 'gender', 'Q36': 'income', 'Q38': 'education', 'Q39': 'ethnic_origin'}
other_data  = [item for item in other_data.items()]
# city_data= [item for item in city_data.items()]
bivariate_analysis_update(data_to_group=data_general_city, main_vars_to_group=city_data, secondary_vars=other_data,
                          field_to_count='level_0',my_data_path=f'{data_path}/by_city')


city
Q2_part	Q6_day_time	Q7_is_locked	Q8_locked_type	Q9_theft_location	Q10_is_regi	Q11_report_loc	Q12_is_insured	Q13_estimate_val	Q14_is_electric	Q15_bike_type	Q17_is_university	Q18_is_recovered	Q19_is_online	Q21_is_police_assist	Q23_recovery_loc	Q24_con_recovery	Q34_1_birth	Q35_gender	Q36_income	Q38_education	Q39_ethnic_origin	

In [66]:
main = data_general[data_general['Q24']=='Repairable– bicycle was damaged and repairs were required before being rideable']
main[main['Q12']=='No ']

,level_0,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,StartDate,EndDate,Status,IPAddress,Progress,...,Q40,Q40_3_TEXT,lat,lon,score,stolen_bikes_place,country,city,state,Q34_1
19,19,21,1470,1556,1556,5/11/2022 19:44,5/11/2022 20:04,IP Address,173.183.182.200,100,...,"A bicycle registry/recovery system (e.g., Bike...",NaN,-113.490140,53.545450,100.0,"Edmonton,Alberta,CAN",CAN,Edmonton,Alberta,1940.0
80,80,82,728,755,755,5/3/2022 21:41,5/3/2022 21:54,IP Address,73.140.210.26,100,...,Email,NaN,-122.514930,47.161950,100.0,"Lakewood,Washington,USA",USA,Lakewood,Washington,1950.0
324,324,326,585,611,611,5/3/2022 13:59,5/3/2022 14:07,IP Address,136.49.97.48,100,...,Email,NaN,-97.742990,30.267590,100.0,"Austin,Texas,USA",USA,Austin,Texas,1960.0
414,414,416,1781,1976,1976,6/1/2022 9:10,6/1/2022 10:44,IP Address,98.171.142.89,100,...,Other (please specify):,Random face book link,-119.699050,34.419380,100.0,"Santa Barbara,California,USA",USA,Santa Barbara,California,1970.0
649,649,651,555,581,581,5/3/2022 13:48,5/3/2022 13:55,IP Address,67.163.88.180,100,...,"A bicycle registry/recovery system (e.g., Bike...",NaN,-111.888220,40.760310,100.0,"Salt Lake City,Utah,USA",USA,Salt Lake City,Utah,1970.0
720,720,722,1696,1836,1836,5/21/2022 5:11,5/22/2022 5:19,IP Address,172.58.28.147,100,...,Email,NaN,-116.510240,33.961450,100.0,"Desert Hot Springs,California,USA",USA,Desert Hot Springs,California,1980.0
731,731,733,417,433,433,4/28/2022 21:38,4/28/2022 21:46,IP Address,70.66.155.124,100,...,"A bicycle registry/recovery system (e.g., Bike...",NaN,-123.365642,48.428409,100.0,"Victoria,British Columbia,CAN",CAN,Victoria,British Columbia,1980.0
750,750,752,1172,1235,1235,5/11/2022 0:59,5/11/2022 1:04,IP Address,80.187.70.182,100,...,"A bicycle registry/recovery system (e.g., Bike...",NaN,-77.031960,38.890370,100.0,"Washington,District of Columbia,USA",USA,Washington,District of Columbia,1980.0
904,904,906,557,583,583,5/3/2022 13:52,5/3/2022 13:56,IP Address,199.241.203.254,100,...,"A bicycle registry/recovery system (e.g., Bike...",NaN,-122.419640,37.777120,100.0,"San Francisco,California,USA",USA,San Francisco,California,1980.0
1054,1054,1056,1731,1891,1891,5/24/2022 18:09,5/24/2022 18:21,IP Address,108.200.200.116,100,...,"A bicycle registry/recovery system (e.g., Bike...",NaN,-90.077750,29.953700,100.0,"New Orleans,Louisiana,USA",USA,New Orleans,Louisiana,1980.0


In [60]:
main

,level_0,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,StartDate,EndDate,Status,IPAddress,Progress,...,Q40,Q40_3_TEXT,lat,lon,score,stolen_bikes_place,country,city,state,Q34_1
